In [1]:
#使用ライブラリ
import pandas as pd
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm

In [2]:
###データ読込(結合したデータ)
train = pd.read_csv(filepath_or_buffer='../j_league/new_data/train_new.csv')
test = pd.read_csv(filepath_or_buffer='../j_league/new_data/test_new.csv')

### 使用変数の作成

In [3]:
#変数選択(目的変数, 気温, 収容人数)
lm_train = train[['y', 'temperature', 'capa']]
lm_test = test[['temperature', 'capa']]

### 重回帰分析

In [4]:
##sklearn を利用して回帰分析
#lm = LinearRegression().fit(lm_train.drop('y', 1), lm_train['y'])

##statsmodels を利用して回帰分析
lm = sm.OLS(lm_train['y'], lm_train.drop('y', 1)).fit()

In [5]:
# サマリーの表示
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.807
Model:                            OLS   Adj. R-squared (uncentered):              0.807
Method:                 Least Squares   F-statistic:                              3595.
Date:                Wed, 12 Aug 2020   Prob (F-statistic):                        0.00
Time:                        12:59:38   Log-Likelihood:                         -17380.
No. Observations:                1721   AIC:                                  3.476e+04
Df Residuals:                    1719   BIC:                                  3.477e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
temperature    13.6159     11.966      1.138      0.255      -9.853      37.085
capa            0.4039      0.009     45.910      0.000       0.387       0.421
==============================================================================
Omnibus:                      123.451   Durbin-Watson:                   1.471
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              322.576
Skew:                           0.394   Prob(JB):                     8.98e-71
Kurtosis:                       4.969   Cond. No.                     2.46e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.46e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### testに当てはめ

In [6]:
pred = lm.predict(lm_test)

### submit形式に整える

In [7]:
out = pd.DataFrame({'id': test['id'],
                   'y': pred})
#CSV出力(ヘッダーなし)
out.to_csv('submit/submit_lm.csv', header=False, index=False)

In [8]:
#出力したCSVが想定どおりのフォーマットか確認する
#先ほど出力したファイルと投稿用のサンプルファイルを読込
submit_sample = pd.read_csv(filepath_or_buffer='../../data/JLeague/sample_submit.csv')
submit_out = pd.read_csv(filepath_or_buffer='submit/submit_lm.csv')

#インデックスを比較
assert len(submit_sample.index.values) == len(submit_out.index.values)
assert (submit_sample.index.values == submit_out.index.values).all()

#1列目がidとなっており、同じIDから始まっているか検証
assert submit_sample.columns[0] == submit_out.columns[0]
assert (submit_out.iloc[:, 0] == submit_sample.iloc[:, 0]).all()

#2列目の列名「y」が入っていないか検証
assert 'y' != submit_out.columns[0]